In [2]:
!pip install wget
import wget
import tarfile
import pandas as pd
import os

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9680 sha256=310c1b97dbe2022daf0b31d6fb02cc68f3f35c7f53a10cfbea5138f4851d44d5
  Stored in directory: /home/ec2-user/.cache/pip/wheels/90/1d/93/c863ee832230df5cfc25ca497b3e88e0ee3ea9e44adc46ac62
Successfully built wget


Matplotlib is building the font cache; this may take a moment.


In [3]:
## Download and extract user/artist dataset ##

filename = 'lastfm-dataset-360K.tar.gz'
source_url = 'http://mtg.upf.edu/static/datasets/last.fm/' + filename
data_dir = 'datasources/raw/'

def getData(source_url, data_dir, filename):
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)
    
    ## Download ##
    wget.download(source_url, out=data_dir)
    
    ## Extract ##
    file = tarfile.open('./' + data_dir + filename)
    file.extractall(data_dir)
    file.close()
    
getData(source_url, data_dir, filename)

In [4]:
## View contents of destination datadir ##
! ls /Users/connorphoenix/ML_Projects/Udacity_MLE__Recommendation/datasources/raw/lastfm-dataset-360K

ls: cannot access /Users/connorphoenix/ML_Projects/Udacity_MLE__Recommendation/datasources/raw/lastfm-dataset-360K: No such file or directory


In [5]:
## Join artist and user files to filter artist plays to users within the United States ##

plays_file = 'usersha1-artmbid-artname-plays.tsv'
users_file = 'usersha1-profile.tsv'
plays_path = './' + data_dir + 'lastfm-dataset-360K/' + plays_file
users_path = './' + data_dir + 'lastfm-dataset-360K/' + users_file

def createDF(plays_path, users_path):
    ''' - Read in and assign column names to artist play and user datasets, 
        - Inner join artist play and user datasets on USER_ID
        - Generate "Continent" column '''
    
    plays_df = pd.read_csv(plays_path, sep='\t', header=None)
    plays_df.columns = ['User_ID', 'Artist_ID', 'Artist', 'Plays']
    print ('Plays DF shape: {}'.format(plays_df.shape))
    
    users_df = pd.read_csv(users_path, sep='\t', header=None)
    users_df.columns = ['User_ID', 'Gender', 'Age', 'Country', 'SignUp_DT']
    users_df = users_df[users_df['Country'] == 'United States']
    print ('Users DF shape: {}'.format(users_df.shape))
    
    fin_df = plays_df.merge(users_df, on ='User_ID', how='inner')  
    fin_df = fin_df[['User_ID', 'Artist', 'Plays']]
    print ('Final DF shape: {}'.format(fin_df.shape))
    
    processed_dir = 'datasources/processed/'
    processed_file = 'user_artist_plays.csv'
    if not os.path.exists(processed_dir):
        os.makedirs(processed_dir)

    fin_df.to_csv('./' + processed_dir + processed_file, index=False)
    
    del plays_df, users_df
    return fin_df

df = createDF(plays_path, users_path)

Plays DF shape: (17535655, 4)
Users DF shape: (67044, 5)
Joined DF shape: (3336272, 3)


In [7]:
## Clean by deleting raw files ##
! rm -rf /home/ec2-user/SageMaker/artist_recommendation_KNN/datasources/raw